In [1]:
import pyliger
import scanpy as sc
import pandas as pd

/mnt/datadisk/lizhongzhan/miniconda3/envs/scSLAT/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
import os 
os.chdir("/mnt/datadisk/lizhongzhan/GLUE/DATA/H3k4me3/")

In [3]:
rna = sc.read_h5ad("rna-pp.h5ad")
# rna.obs_names = ["RNA#"+i for i in list(rna.obs_names)]

In [4]:
# genescore = pd.read_csv("gs.csv", index_col=0)
# genescore = genescore.transpose()
# # genescore.index = ["H3keme3"+i.split(".")[1]+"-1" for i in list(genescore.index)]
# genescore = sc.AnnData(X=genescore)

In [5]:
genescore = sc.read_h5ad("GeneScore.h5ad")
genescore.obs_names = [i.split("-")[0] for i in list(genescore.obs_names)]

In [6]:
cells = [i for i in list(rna.obs_names) if i in list(genescore.obs_names) ]
genescore = genescore[cells,]
genescore

View of AnnData object with n_obs × n_vars = 9542 × 55291
    obs: 'tsse', 'n_fragment', 'frac_dup', 'frac_mito'
    uns: 'reference_sequences'
    obsm: 'insertion'

In [7]:
rna.obs_names = ["RNA#"+i for i in list(rna.obs_names)]
genescore.obs_names = ["GS#"+i for i in list(genescore.obs_names)]

In [8]:
rna.obs.index.name = "cell"
rna.var.index.name = "gene"
genescore.obs.index.name = "cell"
genescore.var.index.name = "gene"
rna.uns["sample_name"] = "RNA"
genescore.uns["sample_name"] = "H3k4me3"

In [9]:
rna.var_names_make_unique()
genescore.var_names_make_unique()

In [10]:
adata_list = [rna, genescore]
ifnb_liger = pyliger.create_liger(adata_list)

Removing 40 genes not expressing in RNA.
Removing 4205 genes not expressing in H3k4me3.


In [11]:
pyliger.normalize(ifnb_liger)
pyliger.select_genes(ifnb_liger)
pyliger.scale_not_center(ifnb_liger)

In [12]:
pyliger.optimize_ALS(ifnb_liger, k = 20)

100%|██████████| 30/30 [06:41<00:00, 13.37s/it]


In [13]:
pyliger.quantile_norm(ifnb_liger)

In [14]:
pyliger.run_umap(ifnb_liger, distance = 'cosine', n_neighbors = 30, min_dist = 0.3)

/mnt/datadisk/lizhongzhan/miniconda3/envs/scSLAT/lib/python3.8/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.


In [15]:
import numpy as np
embed = pd.DataFrame(np.vstack([adata.obsm['H_norm'] for adata in ifnb_liger.adata_list]))

In [16]:
embed.index = sum([list(adata.obs_names) for adata in ifnb_liger.adata_list],[])

In [17]:
pyliger.leiden_cluster(ifnb_liger, resolution=0.7)

In [18]:
tsne_df = ifnb_liger.tsne_coords
tsne_df['Cluster'] = np.asarray(
    np.concatenate([adata.obs['cluster'].to_numpy() for adata in ifnb_liger.adata_list]))
tsne_df['Cluster'] = tsne_df['Cluster'].astype('category')

In [19]:
print(len(set(tsne_df['Cluster'])))

17


In [20]:
umap = pd.DataFrame(tsne_df[["tsne1","tsne2"]])
umap.index = list(embed.index)
cluster = pd.DataFrame(tsne_df["Cluster"])
cluster.index = list(embed.index)

In [21]:
embed.to_csv("rep5/LIGER_embed.csv")
umap.to_csv("rep5/LIGER_umap.csv")
cluster.to_csv("rep5/LIGER_cluster.csv")